## 参考  
### kaggle notebook lightGBM
https://www.kaggle.com/girmdshinsei/for-japanese-beginner-with-wrmsse-in-lgbm?scriptVersionId=31044557

### lightGBM 時系列
https://qiita.com/ground0state/items/657861de619a4e4a30de

### Quick start catboost
https://catboost.ai/docs/concepts/python-quickstart.html

### Pythonでcatboostを使ってみる
#### (cat_featuresの使い方を調べた)
https://qiita.com/shin_mura/items/3d9ce25a60bdd25a3333

### XGBoostパラメータのまとめとランダムサーチ実装
https://qiita.com/FJyusk56/items/0649f4362587261bd57a

In [2]:
import pandas as pd
import os
import gc
import numpy as np
import matplotlib.pyplot as plt

# ラベルエンコーダー
from sklearn import preprocessing, metrics

# lightGBM
import lightgbm as lgb

# CatBoost
from catboost import Pool, CatBoostRegressor

# XGBoost
import xgboost as xgb

# 1セルでまとめて.head()、.tail()等を入力しても大丈夫になる
from IPython.display import display

# 誤差算定
from sklearn.metrics import mean_absolute_error

# 二乗平均平方根誤差 (RMSE)
from sklearn.metrics import mean_squared_error

# 決定係数
from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

# DataFrameの表示数を変更
pd.set_option('max_columns', 500)
pd.set_option('max_rows', 500)

### データ作成

In [4]:
# ローカル用
path = os.getcwd() + "/"

# kaggle Notebook用
INPUT_DIR = '../input/m5-forecasting-accuracy'

calendar.csv -製品の販売日に関する情報が含まれています。
sales_train_validation.csv -製品および店舗ごとの過去の毎日の販売台数データが​​含まれています [d_1 - d_1913]
sample_submission.csv-提出の正しい形式。詳細については、「評価」タブを参照してください。
sell_prices.csv -店舗および日付ごとに販売された製品の価格に関する情報が含まれています。

sales_train_evaluation.csv-締め切りの1か月前に1回ご利用いただけます。売上高が含まれます[d_1 - d_1941]

各行は含むidの連結であるitem_idとstore_idのいずれかである、validation（公共のランキングに対応する）、またはevaluation（プライベートランキングに対応します）。　　 F1-F28各行で販売されるアイテムの28予測日（）を予測しています。　　 以下のためのvalidation行、これに相当するd_1914 - d_1941、とのためevaluationの行、これに相当しますd_1942 - d_1969。　　

validation d_1914 - d_1941の単価と量を予測する。１か月前に１回公開される。 evaluation d_1942-d_1969の単価と量を予測する

### 作成データの読込

In [ ]:
try:
    stv_melt = pd.read_pickle(path + "melt_stv.pkl")
except FileNotFoundError:
    1

### 通常通りcsvを読み込んでデータを作る

In [ ]:
# sales_train_validation.csv
try:
    stv = pd.read_csv(path + "sales_train_validation.csv") # ローカル用
except FileNotFoundError:
    stv = pd.read_csv(f"{INPUT_DIR}/sales_train_validation.csv") # kaggle用

    
# calendar.csv
try:
    cal = pd.read_csv(path + "calendar.csv") # ローカル用
except FileNotFoundError:
    cal = pd.read_csv(f"{INPUT_DIR}/calendar.csv") # kaggle用

    
# sell_prices.csv
try:
    price = pd.read_csv(path + "sell_prices.csv") # ローカル用
except FileNotFoundError:
    price = pd.read_csv(f"{INPUT_DIR}/sell_prices.csv") # kaggle用

    
# sample_submission.csv
try:
    ss = pd.read_csv(path + "sample_submission.csv") # ローカル用
except FileNotFoundError:
    ss = pd.read_csv(f"{INPUT_DIR}/sample_submission.csv") # kaggle用

stv.shape

### ロースペックマシン限定

stv = pd.read_csv(path + "sales_train_validation.csv",
                               skiprows=lambda x: x not in range(0,1001))

cal = pd.read_csv(path + "calendar.csv",
                               skiprows=lambda x: x not in range(0,3001))

price = pd.read_csv(path + "sell_prices.csv",
                               skiprows=lambda x: x not in range(0,3001))

ss = pd.read_csv(path + "sample_submission.csv",
                               skiprows=lambda x: x not in range(0,1001))

In [ ]:
display(stv.head())
display(stv.tail())
display(stv.dtypes)
display(cal.head())
display(cal.tail())
display(cal.dtypes)
display(cal.max())
display(price.head())
display(price.tail())
display(price.dtypes)
display(price.max())
display(price.shape)
display(ss.head())
display(ss.tail())
display(ss.shape)

In [ ]:
day1_1913 = [f"d_{i}" for i in range(1, 1914)]

In [ ]:
stv_melt =  pd.melt(stv, id_vars=['id','store_id','item_id'],
           value_vars=day1_1913,
           var_name = "d", value_name = "vol")

In [ ]:
del day1_1913
gc.collect()

In [ ]:
product = stv[["id","item_id","store_id"]]

In [ ]:
ss_val = ss[0:30490]
ss_val.columns = ["id"] + [f"d_{d}" for d in range(1914, 1942)]

ss_eva = ss[30490:60980]
ss_eva.columns = ["id"] + [f"d_{d}" for d in range(1942, 1970)]

In [ ]:
ss_eva['id'] = ss_eva['id'].str.replace('_evaluation','_validation')

In [ ]:
ss_val = pd.merge(ss_val, product, how = 'left', left_on = ['id'], right_on = ['id'])
ss_eva = pd.merge(ss_eva, product, how = 'left', left_on = ['id'], right_on = ['id'])

In [ ]:
display(ss_val.head(3))
display(ss_val.tail(3))
display(ss_val.shape)
display(ss_eva.head(3))
display(ss_eva.tail(3))
display(ss_eva.shape)

In [ ]:
val_1914_1941 = [f"d_{i}" for i in range(1914, 1942)]
eva_1942_1969 = [f"d_{i}" for i in range(1942, 1970)]

In [ ]:
val_melt =  pd.melt(ss_val, id_vars=['id','store_id', "item_id"],
           value_vars=val_1914_1941,
           var_name = "d", value_name = "vol")
eva_melt =  pd.melt(ss_eva, id_vars=['id','store_id', "item_id"],
           value_vars=eva_1942_1969,
           var_name = "d", value_name = "vol")

In [ ]:
stv_melt = pd.concat([stv_melt, val_melt, eva_melt])

In [ ]:
display(stv_melt.head(3))
display(stv_melt.tail(3))
display(stv_melt.shape)

In [ ]:
del ss, ss_val, ss_eva, val_1914_1941, eva_1942_1969, val_melt, eva_melt, product
gc.collect()

In [ ]:
cal = cal[["date","wm_yr_wk","d","event_name_1","event_type_1","event_name_2","event_type_2"]]

In [ ]:
stv_melt = pd.merge(stv_melt, cal, how = 'left', left_on = ['d'], right_on = ['d'])

In [ ]:
del cal
gc.collect()

In [ ]:
display(stv_melt.head())
display(stv_melt.tail())
display(stv_melt.dtypes)
display(stv_melt.shape)

In [ ]:
stv_melt = stv_melt.merge(price, on = ['store_id', 'item_id', 'wm_yr_wk'], how = 'left')

In [ ]:
del price
gc.collect()

In [ ]:
display(stv_melt.head())
display(stv_melt.tail())
display(stv_melt.dtypes)
display(stv_melt.shape)

### 特徴量作成

In [ ]:
stv_melt["date2"] = pd.to_datetime(stv_melt["date"])

In [ ]:
stv_melt["year"] = stv_melt["date2"].dt.year
stv_melt["month"] = stv_melt["date2"].dt.month
stv_melt["week"] = stv_melt["date2"].dt.week
stv_melt["day"] = stv_melt["date2"].dt.day
stv_melt["dayofweek"] = stv_melt["date2"].dt.dayofweek

stv_melt["year"] = stv_melt["year"].astype('int8')
stv_melt["month"] = stv_melt["month"].astype('int8')
stv_melt["week"] = stv_melt["week"].astype('int8')
stv_melt["day"] = stv_melt["day"].astype('int8')
stv_melt["dayofweek"] = stv_melt["dayofweek"].astype('int8')

In [ ]:
stv_melt = stv_melt.drop("date2", axis=1)

In [ ]:
#　ラグの作成
for i in [7,30,90]:
    stv_melt['shift%s'%i] = stv_melt["vol"].shift(i)

In [ ]:
# 平均値
'''
for i in [7,30,90]:
    stv_melt['mean%s'%i] = stv_melt["vol"].rolling(i).mean()
'''

In [ ]:
# 中央値
'''
for i in [7,30,90]:
    stv_melt['median%s'%i] = stv_melt["vol"].rolling(i).median()
'''

In [ ]:
# 最小値
'''
for i in [7,30,90]:
    stv_melt['min%s'%i] = stv_melt["vol"].rolling(i).min()
'''

In [ ]:
display(stv_melt.head(3))
display(stv_melt.tail(3))
display(stv_melt.dtypes)

In [ ]:
stv_melt["vol"] = stv_melt[["vol"]].astype('int16')
stv_melt["wm_yr_wk"] = stv_melt[ "wm_yr_wk"].astype('int16')

In [ ]:
stv_melt["sell_price"] = stv_melt["sell_price"].astype('float16')
stv_melt["shift7"] = stv_melt["shift7"].astype('float16')

In [ ]:
stv_melt["shift30"] = stv_melt["shift30"].astype('float16')
stv_melt["shift90"] = stv_melt["shift90"].astype('float16')

In [ ]:
# ラベルエンコーダー
lbl = preprocessing.LabelEncoder()
stv_melt["store_id"] = lbl.fit_transform(stv_melt["store_id"])
stv_melt["item_id"] = lbl.fit_transform(stv_melt["item_id"])

stv_melt["store_id"] = stv_melt["store_id"].astype('int8')
stv_melt["item_id"] = stv_melt["item_id"].astype('int8')

In [ ]:
# event は欠損値があるので前処理
stv_melt["event_name_1"] = stv_melt["event_name_1"].fillna("missing", inplace=True)
stv_melt["event_type_1"] = stv_melt["event_type_1"].fillna("missing", inplace=True)
stv_melt["event_name_2"] = stv_melt["event_name_2"].fillna("missing", inplace=True)
stv_melt["event_type_2"] = stv_melt["event_type_2"].fillna("missing", inplace=True)

stv_melt["event_name_1"] = lbl.fit_transform(stv_melt["event_name_1"])
stv_melt["event_type_1"] = lbl.fit_transform(stv_melt["event_type_1"])
stv_melt["event_name_2"] = lbl.fit_transform(stv_melt["event_name_2"])
stv_melt["event_type_2"] = lbl.fit_transform(stv_melt["event_type_2"])

stv_melt["event_name_1"] = stv_melt["event_name_1"].astype('int8')
stv_melt["event_name_2"] = stv_melt["event_name_2"].astype('int8')
stv_melt["event_type_1"] = stv_melt["event_type_1"].astype('int8')
stv_melt["event_type_2"] = stv_melt["event_type_2"].astype('int8')

In [ ]:
display(stv_melt.head(3))
display(stv_melt.tail(3))
display(stv_melt.dtypes)

### 学習用データセットの作成

In [ ]:
x_train = stv_melt[stv_melt['date'] <= '2016-03-27']
y_train = x_train['vol']
x_val   = stv_melt[(stv_melt['date'] > '2016-03-27') & (stv_melt['date'] <= '2016-04-24')]
y_val   = x_val['vol']
test    = stv_melt[(stv_melt['date'] > '2016-04-24')]

In [ ]:
display(test.head())
display(test.tail())
display(test.dtypes)
display(test.shape)

In [ ]:
del stv_melt
gc.collect()

### lightGBM モデルの実行

In [ ]:
features = [
    "store_id",
    "item_id",
    "sell_price",
    "shift7",
    "shift30",
    "shift90",
    "event_name_1",
    "event_type_1",
    "event_name_2",
    "event_type_2",
    "year",
    "month",
    "week",
    "day",
    "dayofweek"
]

In [ ]:
params = {
    'boosting_type': 'gbdt',
    'metric': 'rmse',
    'objective': 'regression',
    'n_jobs': -1,
    'seed': 236,
    'learning_rate': 0.1,
    'bagging_fraction': 0.75,
    'bagging_freq': 10, 
    'colsample_bytree': 0.75
}

In [ ]:
train_set = lgb.Dataset(x_train[features], y_train)
val_set = lgb.Dataset(x_val[features], y_val)

In [ ]:
model_lgb = lgb.train(params,
                      train_set,
                      num_boost_round = 5000,
                      early_stopping_rounds = 50,
                      valid_sets = [train_set, val_set],
                      verbose_eval = 100)

In [ ]:
val_pred = model_lgb.predict(x_val[features])
val_score = np.sqrt(metrics.mean_squared_error(val_pred, y_val))
print(f'Our val rmse score は {val_score}')

In [ ]:
y_pred = model_lgb.predict(test[features])
test['vol'] = y_pred

In [ ]:
predictions = test[['id', 'date', 'vol']]
predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'vol').reset_index()

In [ ]:
display(predictions.head())
display(predictions.tail())
display(predictions.shape)

In [ ]:
del params, model_lgb, y_pred
gc.collect()

In [ ]:
pre_val = predictions.iloc[:,:29]

In [ ]:
pre_eva = pd.concat([predictions.iloc[:,0],predictions.iloc[:,29:57]], axis=1)
pre_eva['id'] = pre_eva['id'].str.replace('_validation', '_evaluation')

In [ ]:
del predictions
gc.collect()

In [ ]:
pre_val.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
pre_eva.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [ ]:
display(pre_val.head())
display(pre_val.tail())
display(pre_val.shape)

display(pre_eva.head())
display(pre_eva.tail())
display(pre_eva.shape)

### 誤差率

In [ ]:
sta = pd.read_csv(path + "sales_train_evaluation.csv")
sta = sta[["d_1914", "d_1915","d_1916","d_1917","d_1918","d_1919","d_1920","d_1921","d_1922","d_1923","d_1924","d_1925","d_1926","d_1927","d_1928","d_1929","d_1930","d_1931","d_1932","d_1933","d_1934","d_1935","d_1936","d_1937","d_1938","d_1939","d_1940","d_1941"]]
sta.columns = ["F1", "F2","F3","F4","F5","F6","F7","F8","F9","F10","F11","F12","F13","F14","F15","F16","F17","F18","F19","F20","F21","F22","F23","F24","F25","F26","F27","F28"]

for i in ["F1", "F2","F3","F4","F5","F6","F7","F8","F9","F10","F11","F12","F13","F14","F15","F16","F17","F18","F19","F20","F21","F22","F23","F24","F25","F26","F27","F28"]:
    sta[i] = sta[i].astype('float64')

In [ ]:
pre_val_temp = pre_val.drop("id",axis=1)

In [ ]:
lgb_score = []

lgb_score.append(np.sqrt(mean_squared_error(sta, pre_val_temp)))
lgb_score.append(mean_absolute_error(sta, pre_val_temp))
lgb_score.append(r2_score(sta, pre_val_temp))

for i in range(3):
    print(lgb_score[i])

f = open('lgb_score.txt', 'w')
for x in lgb_score:
    f.write(str(x) + "\n")
f.close()

In [ ]:
del pre_val_temp, pre_val, lgb_score
gc.collect()

In [ ]:
pre_uni = pd.concat([sta, pre_eva], axis=0)

In [ ]:
pre_uni.to_csv('submission_lgb.csv', index = False)

In [ ]:
del sta, pre_uni
gc.collect()

### catboostの実行

In [ ]:
features = [
    "store_id",
    "item_id",
    "sell_price",
    "shift7",
    "shift30",
    "shift90",
    "event_name_1",
    "event_type_1",
    "event_name_2",
    "event_type_2",
    "year",
    "month",
    "week",
    "day",
    "dayofweek"
]

In [ ]:
# initialize Pool
train_pool = Pool(x_train[features], 
                  y_train)

test_pool = Pool(test[features])

In [ ]:
# specify the training parameters
model2 = CatBoostRegressor(iterations=500,
                          depth=5,
                          learning_rate=0.05,
                          loss_function='RMSE')

In [ ]:
#train the model
model2.fit(train_pool)

In [ ]:
# make the prediction using the resulting model
preds2 = model2.predict(test_pool)

In [ ]:
del model2
gc.collect()

In [ ]:
test2 = test
test2['vol'] = preds2

In [ ]:
predictions2 = test2[['id', 'date', 'vol']]
predictions2 = pd.pivot(predictions2, index = 'id', columns = 'date', values = 'vol').reset_index()

In [ ]:
del train_pool, test_pool, test2, preds2
gc.collect()

In [ ]:
pre_val2 = predictions2.iloc[:,:29]

In [ ]:
pre_eva2 = pd.concat([predictions2.iloc[:,0],predictions2.iloc[:,29:57]], axis=1)
pre_eva2['id'] = pre_eva2['id'].str.replace('_validation', '_evaluation')

In [ ]:
del predictions2
gc.collect()

In [ ]:
pre_val2.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
pre_eva2.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [ ]:
display(pre_val2.head())
display(pre_val2.tail())
display(pre_val2.shape)

display(pre_eva2.head())
display(pre_eva2.tail())
display(pre_eva2.shape)

### 誤差率

In [ ]:
sta = pd.read_csv(path + "sales_train_evaluation.csv")
sta = sta[["d_1914", "d_1915","d_1916","d_1917","d_1918","d_1919","d_1920","d_1921","d_1922","d_1923","d_1924","d_1925","d_1926","d_1927","d_1928","d_1929","d_1930","d_1931","d_1932","d_1933","d_1934","d_1935","d_1936","d_1937","d_1938","d_1939","d_1940","d_1941"]]
sta.columns = ["F1", "F2","F3","F4","F5","F6","F7","F8","F9","F10","F11","F12","F13","F14","F15","F16","F17","F18","F19","F20","F21","F22","F23","F24","F25","F26","F27","F28"]

for i in ["F1", "F2","F3","F4","F5","F6","F7","F8","F9","F10","F11","F12","F13","F14","F15","F16","F17","F18","F19","F20","F21","F22","F23","F24","F25","F26","F27","F28"]:
    sta[i] = sta[i].astype('float64')

In [ ]:
pre_val2_temp = pre_val2.drop("id",axis=1)

In [ ]:
ctb_score = []

ctb_score.append(np.sqrt(mean_squared_error(sta, pre_val2_temp)))
ctb_score.append(mean_absolute_error(sta, pre_val2_temp))
ctb_score.append(r2_score(sta, pre_val2_temp))

for i in range(3):
    print(ctb_score[i])

f = open('ctb_score.txt', 'w')
for x in ctb_score:
    f.write(str(x) + "\n")
f.close()

In [ ]:
del pre_val2_temp, pre_val2, ctb_score
gc.collect()

In [ ]:
pre_uni2 = pd.concat([sta, pre_eva2], axis=0)

In [ ]:
pre_uni2.to_csv('submission_ctb.csv', index = False)

In [ ]:
del sta, pre_uni2, pre_eva2, features
gc.collect()

### XGboost モデルの実行

In [ ]:
xgb_features = [
    "store_id",
    "sell_price",
    "event_name_1",
    "event_type_1",
    "event_name_2",
    "event_type_2",
    "year",
    "month",
    "week",
    "day"
]

In [ ]:
# 学習用のパラメータ
xgb_params = {
        # 回帰問題
        'objective': 'reg:linear',
        # 学習用の指標 (RMSE)
        'eval_metric': 'rmse',
    }
# param['nthread'] = 4
# param['eval_metric'] = 'auc'

In [ ]:
dtrain = xgb.DMatrix(x_train[xgb_features], label=y_train)
dtest = xgb.DMatrix(test[xgb_features], label=test['vol'])
deval= xgb.DMatrix(x_val[xgb_features], label=y_val)

In [ ]:
display(x_train[xgb_features].shape)
display(y_train.shape)
display(x_val[xgb_features].shape)
display(y_val.shape)

In [ ]:
evals = [(dtrain, 'train'), (deval, 'eval')]

In [ ]:
evals_result = {}

In [ ]:
model3 = xgb.train(xgb_params,
                    dtrain,
                    num_boost_round=600,
                    early_stopping_rounds=10,
                    evals=evals,
                    evals_result=evals_result,
                    )

In [ ]:
# 検証用データが各クラスに分類される確率を計算する
y_pred3 = model3.predict(dtest)

In [ ]:
del model3, xgb_params, xgb_features, dtrain, dtest, evals, x_val, y_val, 
gc.collect()

In [ ]:
display(len(y_pred3))
display(test.head())
display(test.shape)

In [ ]:
test3 = test
test3['vol'] = y_pred3

In [ ]:
predictions3 = test3[['id', 'date', 'vol']]
predictions3 = pd.pivot(predictions3, index = 'id', columns = 'date', values = 'vol').reset_index()

In [ ]:
pre_val3 = predictions3.iloc[:,:29]

In [ ]:
pre_eva3 = pd.concat([predictions3.iloc[:,0],predictions3.iloc[:,29:57]], axis=1)
pre_eva3['id'] = pre_eva3['id'].str.replace('_validation', '_evaluation')

In [ ]:
del test3, predictions3, 
gc.collect()

In [ ]:
pre_val3.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]
pre_eva3.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

In [ ]:
display(pre_val3.head())
display(pre_val3.tail())
display(pre_val3.shape)

display(pre_eva3.head())
display(pre_eva3.tail())
display(pre_eva3.shape)

In [ ]:
sta = pd.read_csv(path + "sales_train_evaluation.csv")
sta = sta[["d_1914", "d_1915","d_1916","d_1917","d_1918","d_1919","d_1920","d_1921","d_1922","d_1923","d_1924","d_1925","d_1926","d_1927","d_1928","d_1929","d_1930","d_1931","d_1932","d_1933","d_1934","d_1935","d_1936","d_1937","d_1938","d_1939","d_1940","d_1941"]]
sta.columns = ["F1", "F2","F3","F4","F5","F6","F7","F8","F9","F10","F11","F12","F13","F14","F15","F16","F17","F18","F19","F20","F21","F22","F23","F24","F25","F26","F27","F28"]

for i in ["F1", "F2","F3","F4","F5","F6","F7","F8","F9","F10","F11","F12","F13","F14","F15","F16","F17","F18","F19","F20","F21","F22","F23","F24","F25","F26","F27","F28"]:
    sta[i] = sta[i].astype('float64')

In [ ]:
pre_val3_temp = pre_val3.drop("id",axis=1)

In [ ]:
xgb_score = []

xgb_score.append(np.sqrt(mean_squared_error(sta, pre_val3_temp)))
xgb_score.append(mean_absolute_error(sta, pre_val3_temp))
xgb_score.append(r2_score(sta, pre_val3_temp))

for i in range(3):
    print(xgb_score[i])

f = open('xgb_score.txt', 'w')
for x in xgb_score:
    f.write(str(x) + "\n")
f.close()

In [ ]:
del x_train, y_train, test, pre_val3_temp, pre_val3, xgb_score
gc.collect()

In [ ]:
pre_uni3 = pd.concat([sta, pre_eva3], axis=0)

In [ ]:
pre_uni3.to_csv('submission_xgb.csv', index = False)

In [ ]:
del sta, pre_uni3, pre_eva3
gc.collect()

### データ統合 csv保存

In [41]:
pre_uni  = pd.read_csv(path + "submission_lgb.csv")
pre_uni2 = pd.read_csv(path + "submission_ctb.csv")
pre_uni3 = pd.read_csv(path + "submission_xgb.csv")

In [42]:
pre_uni_am = pre_uni.set_index('id') * 0.1 + pre_uni2.set_index('id') * 0.2 + pre_uni3.set_index('id') * 0.7

In [43]:
pre_uni_am = pre_uni_am.reset_index()
pre_uni_am = pre_uni_am.drop("id", axis=1)
display(pre_uni_am.head())
display(pre_uni_am.columns)

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,2.0,2.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,0.0,0.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,1.0,1.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


Index(['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20', 'F21',
       'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28'],
      dtype='object')

In [44]:
try:
    ss = pd.read_csv(path + "sample_submission.csv") # ローカル用
except FileNotFoundError:
    ss = pd.read_csv(f"{INPUT_DIR}/sample_submission.csv") # kaggle用

ss.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [45]:
pre_uni_am = pd.concat([ss["id"], pre_uni_am], axis=1)
pre_uni_am.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,2.0,2.0,1.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,1.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,0.0,0.0,2.0,2.0,4.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,1.0,1.0,3.0,2.0,3.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [46]:
del pre_uni, pre_uni2, pre_uni3
gc.collect()

44

In [47]:
pre_uni_am.to_csv('submission_uni.csv', index=False)

In [48]:
del pre_uni_am
gc.collect()

44